In [ ]:
%pip install -q transformers torch torchvision torchaudio librosa

In [ ]:
import os

VIDEO_FILE = "video.mp4"

os.system(f"ffmpeg -y -i {VIDEO_FILE} audio.mp3")


In [ ]:
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load model + processor
model_id = "NbAiLabBeta/nb-whisper-medium"
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)

# Load audio (mono, 16kHz)
audio, sr = librosa.load("audio.mp3", sr=16000)

# Whisper input window is ~30s → 30 * 16000 = 480000 samples
max_length = 30 * sr

transcriptions = []

with open("transcription.txt", "w") as f:
    f.write("")

print()
print()

# Split into 30s chunks
for start in range(0, len(audio), max_length):
    chunk = audio[start:start+max_length]

    # Skip empty chunks
    if len(chunk) == 0:
        continue

    # Preprocess
    inputs = processor(
        torch.tensor(chunk),
        sampling_rate=sr,
        return_tensors="pt"
    )

    # Get decoder prompt (Norwegian, transcribe)
    forced_decoder_ids = processor.get_decoder_prompt_ids(
        task="transcribe",
        language="no"
    )

    # Run inference
    with torch.no_grad():
        predicted_ids = model.generate(
            inputs["input_features"],
            forced_decoder_ids=forced_decoder_ids
        )

    # Decode
    text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    transcriptions.append(text)

    with open("transcription.txt", "a") as f:
        f.write(text.strip() + "\n\n")

    print(text + "\n")


with open("transcription-full.txt", "w") as f:
    f.write("\n\n".join(transcriptions))
